In [ ]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/larger_clap_general")
processor = AutoProcessor.from_pretrained("laion/larger_clap_general")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


# Process audio

In [ ]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [ ]:
audio_stimuli = []
stimuli_path = "/content/Exp2/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [ ]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [ ]:
print(audio_embeddings.shape)
# Size of the 2^9 vector

torch.Size([32, 512])


# Process text

In [ ]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]
# What emotion does the sound intend to convey
discrete_captions_perceived = ["I percieve this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
# What emotion do I personally feel in response to listening to the sound
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]
dimensional_captions_perceived = ["I percieve this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I percieve this sound as happiness', 'I percieve this sound as sadness', 'I percieve this sound as anger', 'I percieve this sound as tenderness', 'I percieve this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I percieve this sound as positive', 'I percieve this sound as relaxed', 'I percieve this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [ ]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# Using CLAP processor to tokenize (text-->numerical tokens) and prepare text captions
tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
# Pass processed numerical tokens (from text) through claps text encoder --> text embeddings
tag_embeds = model.get_text_features(**tag_inputs)

In [ ]:
tag_inputs_disc_i = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
tag_embeds_disc_i = model.get_text_features(**tag_inputs_disc_i)

In [ ]:
tag_inputs_dim_p = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_dim_p = model.get_text_features(**tag_inputs_dim_p)

In [ ]:
tag_inputs_disc_p = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_disc_p = model.get_text_features(**tag_inputs_disc_p)

# Generate outputs

In [ ]:
# Similarity scores between audio and text embeddings (dot product)
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([32, 3])


In [ ]:
sims_disc_i = torch.matmul(audio_embeddings, tag_embeds_disc_i.T)
print(sims_disc_i.shape)

torch.Size([32, 5])


In [ ]:
sims_dim_p = torch.matmul(audio_embeddings, tag_embeds_dim_p.T)
print(sims_dim_p.shape)

torch.Size([32, 3])


In [ ]:
sims_disc_p = torch.matmul(audio_embeddings, tag_embeds_disc_p.T)
print(sims_disc_p.shape)

torch.Size([32, 5])


## Load csv files and extract related columns

In [ ]:
IDim_path = '/content/Exp2/Data/Dim/'
#IDim: Indiced dimensional
IDim_responses = []

all_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['Ipositive', 'Irelaxed', 'Iawake', 'stim']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

# Verify we found files
if not all_dfs:
    raise ValueError("No valid CSV files found with the required columns")

# Sort each DataFrame by stimNo
for df in all_dfs:
    df.sort_values('stim', inplace=True)
    df.reset_index(drop=True, inplace=True)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['Ipositive'])
        current_row_relaxeds.append(df.iloc[i]['Irelaxed'])
        current_row_awakes.append(df.iloc[i]['Iawake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 32
Length of relaxed_vector: 32
Length of awake_vector: 32


In [ ]:
PDim_path = '/content/Exp2/Data/Dim/'
#PDim: Percieved dimensional
PDim_responses = []

all_dfs = []

for file in sorted(os.listdir(PDim_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['Ppositive', 'Prelaxed', 'Pawake', 'stim']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

# Verify we found files
if not all_dfs:
    raise ValueError("No valid CSV files found with the required columns")

# Sort each DataFrame by stimNo
for df in all_dfs:
    df.sort_values('stim', inplace=True)
    df.reset_index(drop=True, inplace=True)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['Ppositive'])
        current_row_relaxeds.append(df.iloc[i]['Prelaxed'])
        current_row_awakes.append(df.iloc[i]['Pawake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

PDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(PDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(PDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(PDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 32
Length of relaxed_vector: 32
Length of awake_vector: 32


In [ ]:
IDisc_path = '/content/Exp2/Data/Disc/'
#IDisc: Induced discrete
IDisc_responses = []

all_dfs = []

for file in sorted(os.listdir(IDisc_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["Ihappiness", "Isadness", "Ianger", "Itenderness", "Ifear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

# Verify we found files
if not all_dfs:
    raise ValueError("No valid CSV files found with the required columns")

# Sort each DataFrame by stimNo
for df in all_dfs:
    df.sort_values('stim', inplace=True)
    df.reset_index(drop=True, inplace=True)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['Ihappiness'])
        current_row_sadness.append(df.iloc[i]['Isadness'])
        current_row_anger.append(df.iloc[i]['Ianger'])
        current_row_tenderness.append(df.iloc[i]['Itenderness'])
        current_row_fear.append(df.iloc[i]['Ifear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

IDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(IDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(IDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(IDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(IDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(IDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 32
Length of sadness_vector: 32
Length of anger_vector: 32
Length of tenderness_vector: 32
Length of fear_vector: 32


In [ ]:
PDisc_path = '/content/Exp2/Data/Disc/'
#PDisc: Percieved discrete
PDisc_responses = []

all_dfs = []

for file in sorted(os.listdir(PDisc_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["Phappiness", "Psadness", "Panger", "Ptenderness", "Pfear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

# Verify we found files
if not all_dfs:
    raise ValueError("No valid CSV files found with the required columns")

# Sort each DataFrame by stimNo
for df in all_dfs:
    df.sort_values('stim', inplace=True)
    df.reset_index(drop=True, inplace=True)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['Phappiness'])
        current_row_sadness.append(df.iloc[i]['Psadness'])
        current_row_anger.append(df.iloc[i]['Panger'])
        current_row_tenderness.append(df.iloc[i]['Ptenderness'])
        current_row_fear.append(df.iloc[i]['Pfear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

PDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(PDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(PDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(PDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(PDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(PDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 32
Length of sadness_vector: 32
Length of anger_vector: 32
Length of tenderness_vector: 32
Length of fear_vector: 32


# Evaluate

In [ ]:
# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
# Scaling CLAPs predictions to match the human ratings scale (1-9)
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

Scaled sims shape: torch.Size([32, 3])
Scaled sims (first 5 rows):
tensor([[4.5695, 6.2801, 4.2780],
        [6.2051, 8.4645, 3.8712],
        [4.8703, 5.3706, 5.4857],
        [7.9497, 7.3438, 2.9481],
        [6.6667, 6.0792, 2.0823],
        [5.6881, 4.9882, 2.2049],
        [8.0184, 7.2688, 5.0013],
        [7.4212, 6.2689, 4.2335],
        [4.7882, 7.0018, 5.3951],
        [4.9883, 5.4768, 3.2965]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([32, 3])
Human ratings tensor (first 5 rows):
tensor([[3.7745, 3.7664, 4.6012],
        [5.2729, 5.2924, 5.4271],
        [4.2380, 4.0529, 4.6905],
        [5.4126, 5.0768, 5.9329],
        [4.9408, 4.5192, 6.2842],
        [5.0780, 4.8492, 6.1017],
        [5.1686, 5.1443, 5.3086],
        [4.7716, 4.2137, 6.5511],
        [3.5204, 3.5324, 4.3842],
        [3.9791, 4.2959, 4.7384]])



In [ ]:
# Find the min and max values in the current sims tensor
old_min_disc_i = sims_disc_i.min()
old_max_disc_i = sims_disc_i.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_disc_i = ((sims_disc_i - old_min_disc_i) / (old_max_disc_i - old_min_disc_i)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_disc_i.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_i[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_i.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_i.max():.4f}\n")

human_ratings_tensor_disc_i = torch.tensor([
    IDisc_responses['happiness_vector'],
    IDisc_responses['sadness_vector'],
    IDisc_responses['anger_vector'],
    IDisc_responses['tenderness_vector'],
    IDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_i.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_i[:10]}\n")

Scaled sims shape: torch.Size([32, 5])
Scaled sims (first 5 rows):
tensor([[4.2853, 3.4274, 2.7020, 2.7026, 3.3633],
        [6.5680, 3.4121, 3.1764, 4.9125, 3.2049],
        [4.9814, 4.5133, 5.0213, 5.1574, 5.1848],
        [6.6894, 3.5356, 2.4733, 7.0627, 2.6043],
        [6.4441, 2.6425, 2.4015, 5.2965, 2.7481],
        [5.4315, 3.3213, 1.2940, 5.1715, 2.5574],
        [6.9955, 5.1166, 3.6219, 6.6949, 4.0888],
        [7.8787, 3.3576, 3.7744, 7.8721, 3.3204],
        [5.1161, 4.7742, 4.5332, 3.4050, 4.8818],
        [4.4508, 3.7655, 1.6374, 2.9567, 3.2397]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([32, 5])
Human ratings tensor (first 5 rows):
tensor([[1.6228, 3.2733, 3.7333, 2.1203, 4.5425],
        [2.3532, 3.3734, 2.0583, 2.8818, 2.7857],
        [2.1707, 2.9804, 2.9957, 2.3383, 3.3118],
        [2.6579, 3.1447, 1.8996, 3.1057, 2.6316],
        [2.5830, 2.5961, 1.9534, 3.1324, 2.9382],
        [

In [ ]:
# Find the min and max values in the current sims tensor
old_min_disc_p = sims_disc_p.min()
old_max_disc_p = sims_disc_p.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_disc_p = ((sims_disc_p - old_min_disc_p) / (old_max_disc_p - old_min_disc_p)) * (new_max - new_min) + new_min
print(scaled_sims_disc_p.shape)

print(f"Scaled sims shape: {scaled_sims_disc_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_p.max():.4f}\n")

human_ratings_tensor_disc_p = torch.tensor([
    PDisc_responses['happiness_vector'],
    PDisc_responses['sadness_vector'],
    PDisc_responses['anger_vector'],
    PDisc_responses['tenderness_vector'],
    PDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_p[:10]}\n")

torch.Size([32, 5])
Scaled sims shape: torch.Size([32, 5])
Scaled sims (first 5 rows):
tensor([[4.3286, 5.1867, 2.2010, 3.4601, 4.1826],
        [5.7691, 4.8129, 1.6556, 4.2753, 4.4903],
        [4.9801, 5.7502, 3.2523, 4.5533, 5.5236],
        [6.9503, 5.4685, 1.7783, 6.3583, 5.2940],
        [7.0546, 5.3474, 2.0356, 5.3066, 5.4640],
        [5.9585, 5.6385, 1.4443, 5.3000, 4.8834],
        [7.4203, 6.7566, 2.3336, 6.1585, 6.0861],
        [7.4397, 5.2918, 2.1130, 7.1226, 5.6376],
        [5.5839, 6.7553, 3.8168, 4.1917, 5.7391],
        [5.3800, 6.1482, 2.8052, 3.8173, 4.7590]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([32, 5])
Human ratings tensor (first 5 rows):
tensor([[1.5121, 4.4547, 5.0797, 2.4650, 5.1118],
        [2.5924, 3.8613, 2.3334, 3.3247, 3.3303],
        [1.6533, 4.0592, 4.1332, 3.1161, 4.4234],
        [2.9946, 3.8311, 1.9963, 3.8716, 3.2018],
        [2.8153, 3.8737, 2.4143, 3.5884

In [ ]:
# Find the min and max values in the current sims tensor
old_min_dim_p = sims_dim_p.min()
old_max_dim_p = sims_dim_p.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_dim_p = ((sims_dim_p - old_min_dim_p) / (old_max_dim_p - old_min_dim_p)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_dim_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_dim_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_dim_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_dim_p.max():.4f}\n")

human_ratings_tensor_dim_p = torch.tensor([
    PDim_responses['positive_vector'],
    PDim_responses['relaxed_vector'],
    PDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor_dim_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_dim_p[:10]}\n")

Scaled sims shape: torch.Size([32, 3])
Scaled sims (first 5 rows):
tensor([[2.2598, 3.9521, 5.4222],
        [4.3719, 4.2451, 4.3263],
        [3.0704, 3.5942, 4.5276],
        [5.2338, 4.5582, 4.9855],
        [5.3994, 3.7731, 3.9894],
        [3.9327, 2.3584, 4.0809],
        [5.7798, 3.7842, 4.9103],
        [5.3701, 3.7668, 5.3313],
        [2.6898, 5.6454, 6.5992],
        [3.1058, 4.1421, 4.9972]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([32, 3])
Human ratings tensor (first 5 rows):
tensor([[3.1579, 3.1228, 4.0866],
        [5.3238, 5.2993, 5.1176],
        [4.0297, 3.8703, 4.4003],
        [5.2242, 4.7468, 5.8737],
        [5.0213, 4.4195, 6.4076],
        [5.0674, 4.8129, 5.7620],
        [5.1500, 4.9286, 5.1464],
        [4.8438, 4.1766, 6.6557],
        [3.3632, 3.4957, 4.2013],
        [4.0593, 3.9453, 4.5896]])



 # Comparison Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr # For Pearson correlation

# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
# Average of all absolute errors across all 59 stimuli and 3 dimensions.
# e.g An MAE of 1.5 means CLAP is typically ±1.5 points away from human ratings.
# Mean is calculated over the matrix so one value is outputted
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
mae_disc_i = torch.mean(torch.abs(scaled_sims_disc_i - human_ratings_tensor_disc_i))
mae_disc_p = torch.mean(torch.abs(scaled_sims_disc_p - human_ratings_tensor_disc_p))
mae_dim_p = torch.mean(torch.abs(scaled_sims_dim_p - human_ratings_tensor_dim_p))

print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : {mae:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : {mae_disc_i:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : {mae_disc_p:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : {mae_dim_p:.4f}")

# Comparison Method 2: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).
# Which emotion dimensions does CLAP understand best?

#IDim
correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())

#PDim
correlation_positive_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 0].detach().numpy(), human_ratings_tensor_dim_p[:, 0].numpy())
correlation_relaxed_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 1].detach().numpy(), human_ratings_tensor_dim_p[:, 1].numpy())
correlation_awake_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 2].detach().numpy(), human_ratings_tensor_dim_p[:, 2].numpy())

# IDisc
correlation_happiness, _ = pearsonr(scaled_sims_disc_i[:, 0].detach().numpy(), human_ratings_tensor_disc_i[:, 0].numpy())
correlation_sadness, _ = pearsonr(scaled_sims_disc_i[:, 1].detach().numpy(), human_ratings_tensor_disc_i[:, 1].numpy())
correlation_anger, _ = pearsonr(scaled_sims_disc_i[:, 2].detach().numpy(), human_ratings_tensor_disc_i[:, 2].numpy())
correlation_tenderness, _ = pearsonr(scaled_sims_disc_i[:, 3].detach().numpy(), human_ratings_tensor_disc_i[:, 3].numpy())
correlation_fear, _ = pearsonr(scaled_sims_disc_i[:, 4].detach().numpy(), human_ratings_tensor_disc_i[:, 4].numpy())

#PDisc
correlation_happiness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 0].detach().numpy(), human_ratings_tensor_disc_p[:, 0].numpy())
correlation_sadness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 1].detach().numpy(), human_ratings_tensor_disc_p[:, 1].numpy())
correlation_anger_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 2].detach().numpy(), human_ratings_tensor_disc_p[:, 2].numpy())
correlation_tenderness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 3].detach().numpy(), human_ratings_tensor_disc_p[:, 3].numpy())
correlation_fear_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 4].detach().numpy(), human_ratings_tensor_disc_p[:, 4].numpy())

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:")
print(f"  Positive Dimension: {correlation_positive_dim_p:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed_dim_p:.4f}")
print(f"  Awake Dimension:    {correlation_awake_dim_p:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:")
print(f"  Happiness Dimension: {correlation_happiness:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness:.4f}")
print(f"  Anger Dimension:    {correlation_anger:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness:.4f}")
print(f"  Fear Dimension:    {correlation_fear:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:")
print(f"  Happiness Dimension: {correlation_happiness_disc_p:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness_disc_p:.4f}")
print(f"  Anger Dimension:    {correlation_anger_disc_p:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness_disc_p:.4f}")
print(f"  Fear Dimension:    {correlation_fear_disc_p:.4f}\n")

average_correlation = (correlation_positive + correlation_relaxed + correlation_awake) / 3
average_correlation_disc_i = (correlation_happiness + correlation_sadness + correlation_anger + correlation_tenderness + correlation_fear) / 5
average_correlation_disc_p = (correlation_happiness_disc_p + correlation_sadness_disc_p + correlation_anger_disc_p + correlation_tenderness_disc_p + correlation_fear_disc_p) / 5
average_correlation_dim_p = (correlation_positive_dim_p + correlation_relaxed_dim_p + correlation_awake_dim_p) / 3

print(f"  Average Correlation (IDim): {average_correlation:.4f}")
print(f"  Average Correlation (IDisc): {average_correlation_disc_i:.4f}")
print(f"  Average Correlation (PDisc): {average_correlation_disc_p:.4f}")
print(f"  Average Correlation (PDim): {average_correlation_dim_p:.4f}")

Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : 1.6335
Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : 1.9929
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : 2.3867
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : 1.3077
Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:
  Positive Dimension: 0.7499
  Relaxed Dimension:  0.4970
  Awake Dimension:    -0.5593

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:
  Positive Dimension: 0.7549
  Relaxed Dimension:  0.1421
  Awake Dimension:    -0.3709

Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:
  Happiness Dimension: 0.7000
  Sadness Dimension:  0.0471
  Anger Dimension:    -0.0357
  Tenderness Dimension:    0.5257
  Fear Dimension:    0.2838

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:
  Happiness Dimension: 0.5584


In [ ]:
# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%
# On average, what percentage do CLAP's predictions deviate from human ratings?

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
absolute_percentage_error_disc_i = torch.abs((human_ratings_tensor_disc_i - scaled_sims_disc_i) / human_ratings_tensor_disc_i) * 100
absolute_percentage_error_disc_p = torch.abs((human_ratings_tensor_disc_p - scaled_sims_disc_p) / human_ratings_tensor_disc_p) * 100
absolute_percentage_error_dim_p = torch.abs((human_ratings_tensor_dim_p - scaled_sims_dim_p) / human_ratings_tensor_dim_p) * 100

# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)
mape_disc_i = torch.mean(absolute_percentage_error_disc_i)
mape_disc_p = torch.mean(absolute_percentage_error_disc_p)
mape_dim_p = torch.mean(absolute_percentage_error_dim_p)

print(f"Mean Absolute Percentage Error (MAPE) IDim: {mape:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) IDisc: {mape_disc_i:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDisc: {mape_disc_p:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDim: {mape_dim_p:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))
# How far, on average, are CLAP’s predictions from human ratings—while heavily punishing big errors?

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2
squared_differences_disc_i = (human_ratings_tensor_disc_i - scaled_sims_disc_i)**2
squared_differences_disc_p = (human_ratings_tensor_disc_p - scaled_sims_disc_p)**2
squared_differences_dim_p = (human_ratings_tensor_dim_p - scaled_sims_dim_p)**2

# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)
mse_disc_i = torch.mean(squared_differences_disc_i)
mse_disc_p = torch.mean(squared_differences_disc_p)
mse_dim_p = torch.mean(squared_differences_dim_p)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)
rmse_disc_i = torch.sqrt(mse_disc_i)
rmse_disc_p = torch.sqrt(mse_disc_p)
rmse_dim_p = torch.sqrt(mse_dim_p)

print(f"Root Mean Squared Error (RMSE) IDim: {rmse:.4f}\n")
print(f"Root Mean Squared Error (RMSE) IDisc: {rmse_disc_i:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDisc: {rmse_disc_p:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDim: {rmse_dim_p:.4f}\n")

from sklearn.metrics import r2_score

# Comparison Method 4: R-Squared
# Measures how well CLAP's predictions explain the variance in human ratings.
# What percentage of the changes in human ratings can be predicted by CLAP's model? (1=perfect prediction)

# R-squared IDim
print("\nR-squared scores (IDim):")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print("  valence =", r2_valence)
r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
print("  tension =", r2_tension)
r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
print("  energy =", r2_energy)

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_happiness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 0].numpy(), scaled_sims_disc_i[:, 0].detach().numpy())
print("  happiness =", r2_happiness_disc_i)
r2_sadness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 1].numpy(), scaled_sims_disc_i[:, 1].detach().numpy())
print("  sadness =", r2_sadness_disc_i)
r2_anger_disc_i = r2_score(human_ratings_tensor_disc_i[:, 2].numpy(), scaled_sims_disc_i[:, 2].detach().numpy())
print("  anger =", r2_anger_disc_i)
r2_tenderness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 3].numpy(), scaled_sims_disc_i[:, 3].detach().numpy())
print("  tenderness =", r2_tenderness_disc_i)
r2_fear_disc_i = r2_score(human_ratings_tensor_disc_i[:, 4].numpy(), scaled_sims_disc_i[:, 4].detach().numpy())
print("  fear =", r2_fear_disc_i)

# R-squared PDim
print("\nR-squared scores (PDim):")
r2_valence_dim_p = r2_score(human_ratings_tensor_dim_p[:, 0].numpy(), scaled_sims_dim_p[:, 0].detach().numpy())
print("  valence =", r2_valence_dim_p)
r2_tension_dim_p = r2_score(human_ratings_tensor_dim_p[:, 1].numpy(), scaled_sims_dim_p[:, 1].detach().numpy())
print("  tension =", r2_tension_dim_p)
r2_energy_dim_p = r2_score(human_ratings_tensor_dim_p[:, 2].numpy(), scaled_sims_dim_p[:, 2].detach().numpy())
print("  energy =", r2_energy_dim_p)

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_happiness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 0].numpy(), scaled_sims_disc_p[:, 0].detach().numpy())
print("  happiness =", r2_happiness_disc_p)
r2_sadness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 1].numpy(), scaled_sims_disc_p[:, 1].detach().numpy())
print("  sadness =", r2_sadness_disc_p)
r2_anger_disc_p = r2_score(human_ratings_tensor_disc_p[:, 2].numpy(), scaled_sims_disc_p[:, 2].detach().numpy())
print("  anger =", r2_anger_disc_p)
r2_tenderness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 3].numpy(), scaled_sims_disc_p[:, 3].detach().numpy())
print("  tenderness =", r2_tenderness_disc_p)
r2_fear_disc_p = r2_score(human_ratings_tensor_disc_p[:, 4].numpy(), scaled_sims_disc_p[:, 4].detach().numpy())
print("  fear =", r2_fear_disc_p)

average_r2 = (r2_valence + r2_energy + r2_tension) /3
average_r2_dim_p = (r2_valence_dim_p + r2_energy_dim_p + r2_tension_dim_p) /3
average_r2_disc_i = (r2_happiness_disc_i + r2_sadness_disc_i + r2_anger_disc_i + r2_tenderness_disc_i + r2_fear_disc_i) / 5
average_r2_disc_p = (r2_happiness_disc_p + r2_sadness_disc_p + r2_anger_disc_p + r2_tenderness_disc_p + r2_fear_disc_p) / 5

print("\nAverage R-squared scores:")
print("IDim: ", average_r2)
print("PDim: ", average_r2_dim_p)
print("IDisc: ", average_r2_disc_i)
print("PDisc: ", average_r2_disc_p)

Mean Absolute Percentage Error (MAPE) IDim: 33.60%
Mean Absolute Percentage Error (MAPE) IDisc: 79.76%
Mean Absolute Percentage Error (MAPE) PDisc: 87.88%
Mean Absolute Percentage Error (MAPE) PDim: 29.27%

Root Mean Squared Error (RMSE) IDim: 2.0180

Root Mean Squared Error (RMSE) IDisc: 2.4475

Root Mean Squared Error (RMSE) PDisc: 2.8074

Root Mean Squared Error (RMSE) PDim: 1.7266


R-squared scores (IDim):
  valence = -2.238218307495117
  tension = -2.197026014328003
  energy = -9.326789855957031

R-squared scores (IDisc):
  happiness = -31.431941986083984
  sadness = -18.251853942871094
  anger = -5.387155532836914
  tenderness = -14.908712387084961
  fear = -3.457709789276123

R-squared scores (PDim):
  valence = -0.30391860008239746
  tension = -2.4049270153045654
  energy = -3.8642449378967285

R-squared scores (PDisc):
  happiness = -21.28409767150879
  sadness = -22.638193130493164
  anger = -1.755206823348999
  tenderness = -9.14949893951416
  fear = -3.7326302528381348

Av